In [1]:
# 程序功能：爬取小红书任意笔记下的评论
import requests
from time import sleep
import pandas as pd
import os
import time
import datetime
import random

In [2]:
# 请求头
h1 = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    # 需要换成自己的cookie
    #页面检查，network，fetch/XHR,点展开评论会有page?note_id=,header会显示cookie
    'Cookie': 'a1=189ab7d6827p13w1fs64lhnymykubzghyzyr8g01b30000277987; webId=527fe3a0a48ac874bbb85e38bc637889; gid=yYj0DWfKffvjyYj0DWfKYuk8JWUyqEyiMK41vyDxIv6vTFq87DukxY888JWWjYW8jf8SJJy4; abRequestId=527fe3a0a48ac874bbb85e38bc637889; webBuild=3.15.3; web_session=0400698f0580fa56634af27a7a374b4074fe99; xsecappid=xhs-pc-web; unread={%22ub%22:%22654e612600000000330014af%22%2C%22ue%22:%226547bfe7000000001e03f3a8%22%2C%22uc%22:16}; websectiga=3fff3a6f9f07284b62c0f2ebf91a3b10193175c06e4f71492b60e056edcdebb2; sec_poison_id=d06bb14b-a99a-453b-89b3-e8793fb8c2fd',
}


In [3]:
def trans_date(v_timestamp):
    """13位时间戳转换为时间字符串"""
    v_timestamp = int(str(v_timestamp)[:10])
    timeArray = time.localtime(v_timestamp)
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    return otherStyleTime


In [4]:
# 当前时间戳
now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
# 目标笔记id（找了5个"消费降级"相关的笔记）
note_id_list = ['62288e92000000000102bb1d', '64f1ad9c0000000020001c8a','654de10c000000003202ddf6', 
                '652d2a470000000022029dfc', '64cb0a6a000000000c034120','6558c17b000000003200ad6e',
               '649c2c3000000000130006e5','64a6289a000000003100ae84','65572a9000000000170342c4',
               '64c4bdb5000000000c03537f']
# 保存文件名
result_file = '小红书评论_{}.csv'.format(now)



In [5]:
for note_id in note_id_list:
	page = 1
	while True:
		if page == 1:
			url = 'https://edith.xiaohongshu.com/api/sns/web/v2/comment/page?note_id={}&top_comment_id=&image_scenes=FD_WM_WEBP,CRD_WM_WEBP'.format(
				note_id)
		else:
			url = 'https://edith.xiaohongshu.com/api/sns/web/v2/comment/page?note_id={}&top_comment_id=&image_scenes=FD_WM_WEBP,CRD_WM_WEBP&cursor={}'.format(
				note_id, next_cursor)
		# 发送请求
		r = requests.get(url, headers=h1)
		print('r.status_code：', r.status_code)
		if str(r.status_code) != '200':
			continue
		# 以json格式接收数据
		json_data = r.json()
		# 定义一些空列表用于存放数据
		content_list = []  # 评论内容
		create_time_list = []  # 评论时间
		ip_list = []  # 评论IP属地
		like_count_list = []  # 评论点赞数
		nickname_list = []  # 评论者昵称
		user_id_list = []  # 评论者id
		user_link_list = []  # 评论者主页链接
		comment_level_list = []  # 评论级别
		comment_num = len(json_data['data']['comments'])
		print('开始爬【{}】第{}页，笔记数量：{}'.format(note_id, page, comment_num))
		sleep(random.uniform(0, 2))
		# 循环解析json数据
		print('开始爬取【一级】评论')
		for c in json_data['data']['comments']:
			# 评论者昵称
			nickname = c['user_info']['nickname']
			nickname_list.append(nickname)
			# 评论者id
			user_id = c['user_info']['user_id']
			user_id_list.append(user_id)
			# 评论者主页链接
			user_link = 'https://www.xiaohongshu.com/user/profile/' + user_id
			user_link_list.append(user_link)
			# 评论时间
			create_time = trans_date(c['create_time'])
			create_time_list.append(create_time)
			# 评论IP属地
			try:
				ip = c['ip_location']
			except:
				ip = ''
			ip_list.append(ip)
			# 评论点赞数
			like_count = c['like_count']
			like_count_list.append(like_count)
			# 评论内容
			content = c['content']
			content_list.append(content)
			# 二级评论
			sub_comment_count = c['sub_comment_count']
			# 根评论id
			root_comment_id = c['id']
			# 评论级别
			comment_level_list.append('根评论')
			if int(sub_comment_count) > 0:
				print('开始爬取【二级】评论')
				for c2 in c['sub_comments']:
					# 评论者昵称
					nickname2 = c2['user_info']['nickname']
					nickname_list.append(nickname2)
					# 评论者id
					user_id2 = c2['user_info']['user_id']
					user_id_list.append(user_id2)
					# 评论者主页链接
					user_link2 = 'https://www.xiaohongshu.com/user/profile/' + user_id2
					user_link_list.append(user_link2)
					# 评论时间
					create_time2 = trans_date(c2['create_time'])
					create_time_list.append(create_time2)
					# 评论IP属地
					try:
						ip2 = c2['ip_location']
					except:
						ip2 = ''
					ip_list.append(ip2)
					# 评论点赞数
					like_count2 = c2['like_count']
					like_count_list.append(like_count2)
					# 评论内容
					content2 = c2['content']
					content_list.append(content2)
					# 评论级别
					comment_level_list.append('二级评论')
			# 展开评论
			if c['sub_comment_has_more'] == True:
				print('开始爬取【二级展开】评论')
				url_more = 'https://edith.xiaohongshu.com/api/sns/web/v2/comment/sub/page?note_id={}&root_comment_id={}&image_scenes=FD_WM_WEBP,CRD_WM_WEBP&cursor={}&num=10'.format(
					note_id, root_comment_id, c['sub_comment_cursor'])
				r_more = requests.get(url_more, headers=h1)
				print('r_more status_code:', r_more.status_code)
				json_data_more = r_more.json()
				# 循环解析json数据
				for c_more in json_data_more['data']['comments']:
					# 评论者昵称
					nickname_more = c_more['user_info']['nickname']
					nickname_list.append(nickname_more)
					# 评论者id
					user_id_more = c_more['user_info']['user_id']
					user_id_list.append(user_id_more)
					# 评论者主页链接
					user_link_more = 'https://www.xiaohongshu.com/user/profile/' + user_id_more
					user_link_list.append(user_link_more)
					# 评论时间
					create_time_more = trans_date(c_more['create_time'])
					create_time_list.append(create_time_more)
					# 评论IP属地
					try:
						ip_more = c_more['ip_location']
					except:
						ip_more = ''
					ip_list.append(ip_more)
					# 评论点赞数
					like_count_more = c_more['like_count']
					like_count_list.append(like_count_more)
					# 评论内容
					content_more = c_more['content']
					content_list.append(content_more)
					# 评论级别
					comment_level_list.append('二级展开评论')
		# 保存数据到DF
		df = pd.DataFrame(
			{
				'笔记链接': 'https://www.xiaohongshu.com/explore/' + note_id,
				'页码': page,
				'评论者昵称': nickname_list,
				'评论者id': user_id_list,
				'评论者主页链接': user_link_list,
				'评论时间': create_time_list,
				'评论IP属地': ip_list,
				'评论点赞数': like_count_list,
				'评论级别': comment_level_list,
				'评论内容': content_list,
			}
		)
		# 设置csv文件表头
		if os.path.exists(result_file):
			header = False
		else:
			header = True
		# 保存到csv
		df.to_csv(result_file, mode='a+', header=header, index=False, encoding='utf_8_sig')
		print('文件保存成功：', result_file)
		
		if not json_data['data']['has_more']:
			print('没有下一页了，终止循环！')
			break
		# 判断终止条件
		next_cursor = json_data['data']['cursor']
		page += 1

r.status_code： 200
开始爬【62288e92000000000102bb1d】第1页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【62288e92000000000102bb1d】第2页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【62288e92000000000102bb1d】第3页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【62288e92000000000102bb1d】第4页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【62288e92000000000102bb1d】第5页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【62288e92000000000102bb1d】第6页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.c

开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64f1ad9c0000000020001c8a】第27页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64f1ad9c0000000020001c8a】第28页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64f1ad9c0000000020001c8a】第29页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64f1ad9c0000000020001c8a】第30页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64f1ad9c0000000020001c8a】第31页，笔记数量：10
开始爬取【一级】评论
开始爬

r.status_code： 200
开始爬【654de10c000000003202ddf6】第35页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【654de10c000000003202ddf6】第36页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【654de10c000000003202ddf6】第37页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【654de10c000000003202ddf6】第38页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【654de10c000000003202ddf6】第39页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【654de10c000000003202ddf6】第40页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成

r.status_code： 200
开始爬【6558c17b000000003200ad6e】第1页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【6558c17b000000003200ad6e】第2页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【6558c17b000000003200ad6e】第3页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二

r.status_code： 200
开始爬【649c2c3000000000130006e5】第20页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】第21页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】第22页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】第23页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】第24页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】第25页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】第26页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【649c2c3000000000130006e5】

...

r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1263页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1264页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1265页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1266页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1267页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1268页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1269页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1270页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1271页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.st

r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1335页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1336页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1337页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1338页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1339页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1340页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1341页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1342页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 20

开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1407页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1408页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1409页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1410页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1411页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1412页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1413页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1414页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.st

开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1479页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1480页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1481页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1482页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1483页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1484页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1485页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 

开始爬取【一级】评论
开始爬取【二级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1549页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1550页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1551页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1552页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1553页，笔记数量：10
开始爬取【一级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级】评论
开始爬取【二级展开】评论
r_more status_code: 200
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1554页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb5000000000c03537f】第1555页，笔记数量：10
开始爬取【一级】评论
文件保存成功： 小红书评论_20231202155053.csv
r.status_code： 200
开始爬【64c4bdb